In [1]:
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# 1. Analyse de l’évolution de la distance Terre–Lune
# ============================================================

# ------------------------------------------------------------
# 1.1 Chargement et affichage des données
# ------------------------------------------------------------

# 1) Charger les données stockées dans le fichier
data = np.load("distance_earth_moon_2011.npz")
distances = data["distances"]       # distances Terre–Lune (en km)
K = len(distances)                  # nombre de points temporels

print("Nombre d'échantillons K :", K)

# 2) Calculer la distance Terre–Lune moyenne
mean_distance = np.mean(distances)
print("Distance Terre-Lune moyenne ≈", mean_distance, "km")
# Remarque : pour ces données, on obtient environ 385073 km.

# 3) Afficher l’évolution de la distance Terre–Lune en fonction des jours
jours = np.arange(K)   # un échantillon par jour

plt.figure()
plt.plot(jours, distances)
plt.xlabel("Jour")
plt.ylabel("Distance Terre-Lune (km)")
plt.title("Évolution de la distance Terre-Lune en 2011")
plt.grid(True)
plt.tight_layout()
plt.show()


# ------------------------------------------------------------
# 1.2 Analyse fréquentielle
# ------------------------------------------------------------

# 1) Unité de l’axe fréquentiel
# ------------------------------------------------------------
# Réponse théorique :
# Le signal est échantillonné une fois par jour, donc :
#   - l’unité temporelle est le jour
#   - la période d’échantillonnage Te = 1 jour
#   - la fréquence d’échantillonnage Fe = 1 / Te = 1 échantillon/jour
#
# Par conséquent, l’axe fréquentiel est exprimé en :
#   -> cycles par jour (ou jour^-1).


# 2) Spectre 4K-points sur les fréquences positives
# ------------------------------------------------------------

Fe = 1.0              # 1 échantillon par jour
N = 4 * K             # FFT sur 4K points (zéro-padding)

# FFT du signal brut (avec composante continue)
X = np.fft.fft(distances, n=N)
freqs = np.fft.fftfreq(N, d=1/Fe)   # axe fréquentiel en cycles/jour

# On ne conserve que les fréquences positives
mask_pos = freqs >= 0
freqs_pos = freqs[mask_pos]
X_pos = X[mask_pos]

plt.figure()
plt.plot(freqs_pos, np.abs(X_pos))
plt.xlabel("Fréquence (cycles/jour)")
plt.ylabel("Amplitude")
plt.title("Spectre 4K points (fréquences positives, signal brut)")
plt.grid(True)
plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# Commentaire (1.2.2) :
# - Le pic principal du spectre se trouve à f = 0 (fréquence nulle).
#   Il correspond à la composante continue du signal, c’est-à-dire
#   à la distance Terre–Lune moyenne.
# - Son amplitude est proportionnelle à K * distance_moyenne.
#
# Méthode pour l’éliminer :
#   -> On soustrait la valeur moyenne du signal temporel :
#      s_c[n] = distances[n] - mean_distance
#   puis on recalcule la FFT. Cela supprime la composante à f = 0.
# ------------------------------------------------------------

# Signal centré (sans composante continue)
signal_centre = distances - mean_distance

Xc = np.fft.fft(signal_centre, n=N)
Xc_pos = Xc[mask_pos]


# 3) Spectre 4K en échelle logarithmique sans pic principal
# ------------------------------------------------------------

plt.figure()
plt.semilogy(freqs_pos, np.abs(Xc_pos))
plt.xlabel("Fréquence (cycles/jour)")
plt.ylabel("Amplitude (échelle logarithmique)")
plt.title("Spectre 4K points (log) sans composante continue")
plt.grid(True)
plt.tight_layout()
plt.show()

# On peut relever numériquement les fréquences des pics
mag = np.abs(Xc_pos)
# Normalisation pour détecter les pics significatifs
mag_norm = mag / np.max(mag)
threshold = 0.1  # seuil arbitraire (10% du max)
indices_significatifs = np.where(mag_norm > threshold)[0]
freqs_significatives = freqs_pos[indices_significatifs]

print("Fréquences significatives (cycles/jour) :")
print(freqs_significatives)

# ------------------------------------------------------------
# Commentaire (1.2.3) :
# En observant le spectre (ou les valeurs imprimées), on trouve
# des pics principaux autour de fréquences telles que :
#   f1 ≈ 0.036 cycles/jour  -> T1 ≈ 1 / f1 ≈ 27.5 jours
#   f2 ≈ 0.032 cycles/jour  -> T2 ≈ 32   jours
#   f3 ≈ 0.068 cycles/jour  -> T3 ≈ 14.8 jours
#
# Ces fréquences correspondent aux composantes périodiques
# de l’évolution de la distance Terre–Lune.
# ------------------------------------------------------------


# 4) Analyse avec fenêtrage du signal temporel
# ------------------------------------------------------------
# On choisit une fenêtre de Hann :
# - bon compromis entre réduction des fuites spectrales
#   (lobes secondaires) et résolution fréquentielle.
# - adaptée à l’analyse de signaux quasi périodiques sur une
#   fenêtre finie d’observation.

fenetre = np.hanning(K)
signal_fenetre = signal_centre * fenetre

Xw = np.fft.fft(signal_fenetre, n=N)
Xw_pos = Xw[mask_pos]

plt.figure()
plt.semilogy(freqs_pos, np.abs(Xw_pos))
plt.xlabel("Fréquence (cycles/jour)")
plt.ylabel("Amplitude (échelle logarithmique)")
plt.title("Spectre 4K (log) avec fenêtre de Hann")
plt.grid(True)
plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# Commentaire (1.2.4) :
# - Le fenêtrage réduit les lobes secondaires et rend les pics
#   de fréquence plus nets.
# - On identifie mieux les composantes fréquentielles
#   principales, au prix d’un léger élargissement des pics.
# - Dans ce contexte, le fenêtrage permet donc de mieux
#   caractériser les périodes principales de variation de la
#   distance Terre–Lune.
# ------------------------------------------------------------


# 5) Fréquence maximale significative et Fe minimale
# ------------------------------------------------------------

# On utilise les fréquences significatives déjà détectées
f_max = np.max(freqs_significatives)
Fe_min = 2 * f_max          # critère de Nyquist
Te_max = 1.0 / Fe_min       # période d’échantillonnage maximale

print("Fréquence maximale significative f_max ≈", f_max, "cycles/jour")
print("Fréquence d'échantillonnage minimale Fe_min ≈", Fe_min, "échantillons/jour")
print("Période d'échantillonnage maximale Te_max ≈", Te_max, "jours")

# ------------------------------------------------------------
# Commentaire (1.2.5) :
# - La fréquence maximale significative est de l’ordre de
#   f_max ≈ 0.068 cycles/jour.
# - Pour éviter le repliement spectral, il faut :
#       Fe ≥ 2 * f_max  ≈ 0.136 échantillons/jour
#   soit une mesure environ tous les 7–8 jours.
# - Dans les données réelles, l’échantillonnage est d’un
#   échantillon par jour, donc largement suffisant pour
#   éviter le repliement spectral.
# ------------------------------------------------------------


# 6) Caractéristiques de l’évolution de la distance Terre–Lune
# ------------------------------------------------------------

# On peut trouver la fréquence dominante (hors composante DC)
idx_max = np.argmax(mag)            # sur le spectre centré (signal_centre)
f_dom = freqs_pos[idx_max]
T_dom = 1.0 / f_dom

print("Fréquence dominante ≈", f_dom, "cycles/jour")
print("Période dominante ≈", T_dom, "jours")

# ------------------------------------------------------------
# Commentaire (1.2.6) :
# - La fréquence dominante correspond à une période d’environ
#   T_dom ≈ 27–28 jours, ce qui reflète le cycle principal
#   d’orbite de la Lune autour de la Terre.
# - D’autres pics autour de ~15 jours et ~30 jours traduisent
#   des composantes supplémentaires (harmoniques, effets liés
#   à l’excentricité de l’orbite, etc.).
# - Globalement, la distance Terre–Lune varie de façon
#   périodique, avec une composante principale d’environ
#   27–28 jours, modulée par des composantes secondaires.
# ------------------------------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: 'distance_earth_moon_2011.npz'